# TP3 : Dynamic programming for the knapsack problem


### Reading instances

In [1]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Dynamic programming : replace all TODO of the code

In [7]:
function solveKnapDP(filename)

    ## Loading data ##
    prices, weights, capacity_bag = readKnaptxtInstance(filename)

    ## Initialisation of the matrix ##
    n_items = length(prices)
    n_columns = capacity_bag + 1
    matrix_best_price = zeros(n_items, n_columns)

    ## Fulfilling th first row ##
    for capacity in weights[1] + 1:capacity_bag + 1
        matrix_best_price[1,capacity] = prices[1]
    end
    
    ## Fulfilling the matrix with the recursive formula ##
    for item in 2:n_items
        for capacity in 1:n_columns
            if capacity >= weights[item]+1
                matrix_best_price[item, capacity] = max(matrix_best_price[item - 1, capacity - weights[item]] + prices[item], matrix_best_price[item - 1, capacity])
            else
                matrix_best_price[item, capacity] = matrix_best_price[item - 1, capacity]
            end
        end
    end

    best_cost = maximum(matrix_best_price)
    println("Best cost = ", best_cost)

    ## Backtracking to identify items selected
    remaining_capacity = capacity_bag + 1 
    list_items_selected = []

    ## Backtracking rows 2:n_items
    for item in n_items:-1:2
        if matrix_best_price[item, remaining_capacity] != matrix_best_price[item-1, remaining_capacity]
            push!(list_items_selected, item)
            remaining_capacity -= weights[item]
        end
    end

    ## Checking whether item n°1 has been selected 
    if matrix_best_price[1,remaining_capacity] != matrix_best_price[1, 1]
        push!(list_items_selected, 1)
    end

    println("items selected : ", reverse(list_items_selected))
    return best_cost, reverse(list_items_selected)
end


println("\n--- Test opb ---")
time_ms = @elapsed solveKnapDP("InstancesKnapSack/test.opb.txt")
println(round(time_ms * 1000, digits=2), " ms")
#solveKnapDP("InstancesKnapSack/test.opb.txt") 

instances = [
    "InstancesKnapSack/Similar_Weights/KnapSack_100_1000_-995.opb.txt",
    "InstancesKnapSack/Weakly_Correlated/KnapSack_100_1000_-1514.opb.txt",
    "InstancesKnapSack/Strongly_Correlated/KnapSack_100_1000_-2397.opb.txt",
    "InstancesKnapSack/Uncorrelated/KnapSack_100_1000_-9147.opb.txt"
]

println("\n--- Tests KnapScak ---")
for i in instances 
    print("Instance: ", split(i, "/")[end], " => ")
    time_ms = @elapsed solveKnapDP(i)
    println(round(time_ms * 1000, digits=2), " ms")
end


--- Test opb ---
Best cost = 65.0
items selected : Any[2, 4]
114.16 ms

--- Tests KnapScak ---
Instance: KnapSack_100_1000_-995.opb.txt => Best cost = 995.0
items selected : Any[29]
2769.93 ms
Instance: KnapSack_100_1000_-1514.opb.txt => Best cost = 1514.0
items selected : Any[11, 24, 33, 38, 45, 49, 57, 71, 85]
33.87 ms
Instance: KnapSack_100_1000_-2397.opb.txt => Best cost = 2397.0
items selected : Any[2, 13, 21, 27, 30, 47, 51, 65, 71, 75, 77, 86, 90, 97]
33.71 ms
Instance: KnapSack_100_1000_-9147.opb.txt => Best cost = 9147.0
items selected : Any[7, 11, 14, 24, 26, 31, 33, 38, 39, 49, 54, 61]
33.21 ms


## Comparaison TP2 (Branch-and-Bound) vs TP3 (Programmation Dynamique)

### Rappel des approches

| Critère | TP2 — Branch-and-Bound | TP3 — Programmation Dynamique |
|---------|------------------------|------------------------------|
| **Principe** | Exploration intelligente d'un arbre de décision avec coupes (bornes) | Construction d'une matrice de sous-problèmes optimaux |
| **Complexité temporelle** | Exponentielle dans le pire cas, mais réduite par les coupes | Pseudo-polynomiale : O(n × C) où C = capacité |
| **Complexité spatiale** | O(n) pour la pile récursive + arbre mémorisé | O(n × C) pour la matrice |
### Résultats sur l'instance test.opb.txt (4 objets, capacité 10)

| Méthode | Valeur optimale | Solution |
|---------|-----------------|----------|
| **Branch-and-Bound initial (TP2)** | 65 | {2, 4} |
| **Branch-and-Bound amélioré (TP2)** | 65 | {2, 4} |
| **Programmation Dynamique (TP3)** | 65 | {2, 4} |

#### Avantages de la Programmation Dynamique (TP3)
- **Temps déterministe** : la complexité O(n × C) est garantie, pas de variabilité selon l'instance.
- **Simplicité** : pas besoin de calcul de bornes, de tests de sondabilité ou de stratégies d'exploration.
- **Performance** : très efficace quand la capacité C est "raisonnable" (entiers pas trop grands).

#### Avantages du Branch-and-Bound (TP2)
- **Mémoire** : ne stocke que le chemin courant (O(n)), tandis que la programmation dynamique stocke une matrice O(n × C).
- **Grandes capacités** : si C est très grand, le temps de calcul de la programmation dynamique explose, mais le Branch-and-Bound peut fonctionner si les coupes sont efficaces.

#### Résultats TP2 sur instances de taille 100

| Instance | Noeuds (init) | Temps init (ms) | Noeuds (amélioré) | Temps amél. (ms) | Speedup |
|----------|---------------|-----------------|-------------------|------------------|---------|
| Similar Weights | 1 113 | 164 | 199 | 13 | 12.7× |
| Weakly Correlated | 36 265 | 2 494 | 973 | 98 | 25.6× |
| Strongly Correlated | 80 849 | 3 354 | 1 281 | 90 | 37.4× |
| Uncorrelated | 17 771 | 963 | 745 | 52 | 18.4× |

**Moyenne speedup Branch-and-Bound amélioré vs initial : 19×**

#### Tableau des Résultats Croisés

| Instance (n=100) | Temps TP2 (Branch-and-Bound Amélioré) | Temps TP3 (Programmation dynamique) | Vainqueur |
| :--- | :--- | :--- | :--- |
| Similar Weights | 13 ms | ~2716 ms | TP2 |
| Weakly Correlated | 98 ms | ~32 ms | TP3 |
| Strongly Correlated | 90 ms | ~31 ms | TP3 |
| Uncorrelated | 52 ms | ~31 ms | TP3 |


### Quand utiliser quelle méthode ?

| Situation | Méthode recommandée |
|-----------|---------------------|
| Capacité C moyenne | **Programmation Dynamique** (TP3) — plus simple et rapide |
| Capacité C grande | **Branch-and-Bound** (TP2) — évite l'explosion mémoire |

### Conclusion

Les deux méthodes garantissent la solution optimale, mais leurs performances dépendent des caractéristiques de l'instance :
- **TP3 (Programmation dynamique)** est le choix par défaut pour le sac à dos classique avec capacités entières raisonnables.
- **TP2 (Branch-and-Bound amélioré)** brille sur des instances où la structure permet de nombreuses coupes, et reste applicable quand la capacité est trop grande pour la programmation dynamique.